In [2]:
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import string
import numpy as np
import pandas as pd
from numpy import array
from PIL import Image
import pickle

import matplotlib.pyplot as plt
import keras
import sys, time, os, warnings
warnings.filterwarnings("ignore")
import re

import tensorflow as tf
from tqdm import tqdm

from nltk.translate.bleu_score import sentence_bleu

from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, BatchNormalization
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import add
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [4]:
import pandas as pd

# Specify the path to the Stanford Paragraph Captioning dataset file
dir_Stanford_text = '/kaggle/input/standford-paragraph-nepali-dataset/Caption_final.csv'

# Read the Stanford Paragraph Captioning dataset
data = pd.read_csv(dir_Stanford_text, delimiter=',')

# Remove unwanted rows if needed
# data = data[data['train'] == 'TRUE']  # Adjust as needed
print(data.columns)

# Reorganize the columns
data = data[['Image_name','Paragraph']]

# Display the first few rows of the DataFrame
data.head(2)

Index(['Image_name', 'Paragraph', 'train', 'test', 'val'], dtype='object')


,Image_name,Paragraph
0,2356347,यसको अगाडि झ्यालहरूमा बारहरू भएको ठूलो भवन। भव...
1,2317429,एउटा सेतो गोलो प्लेट एउटा टेबलमा छ जसमा प्लास...


In [5]:
vocabulary = []
for txt in data.Paragraph.values:
    vocabulary.extend(txt.split())
print('Vocabulary Size: %d' % len(set(vocabulary)))

Vocabulary Size: 27661


In [6]:
img = data["Image_name"].tolist()
caption = data["Paragraph"].tolist()

In [7]:
# print(img[-3:])
# print(caption[:3])
print(f"len(img) : {len(img)}")
print(f"len(caption) : {len(caption)}")

len(img) : 20359
len(caption) : 20359


In [8]:
import string

# To remove punctuations
def remove_punctuation(text_original):
    translator = str.maketrans('', '', string.punctuation)
    text_no_punctuation = text_original.translate(translator)
    return text_no_punctuation

# To remove single characters
def remove_single_character(text):
    words = text.split()
    text_len_more_than1 = " ".join([word for word in words if len(word) > 1])
    return text_len_more_than1

# To remove numeric values
def remove_numeric(text):
    words = text.split()
    text_no_numeric = " ".join([word for word in words if not word.isdigit()])
    return text_no_numeric


In [9]:
def text_clean(text_original):
    text = remove_punctuation(text_original)
    text = remove_single_character(text)
    text = remove_numeric(text)
    return(text)

text = '। छोटो, टपलेस, प्लास्टिक कadfन्टेन200रम?ा सेतो ..प्लेटको छेउ//माको टाउको माथि हरियो टेनिस बल छ।'
print(text_clean(text))

for i, caption in enumerate(data.Paragraph.values):
    newcaption = text_clean(caption)
    data["Paragraph"].iloc[i] = newcaption

छोटो टपलेस प्लास्टिक कadfन्टेन200रमा सेतो प्लेटको छेउमाको टाउको माथि हरियो टेनिस बल छ।


In [10]:
data['Paragraph'][10]

'सेतो प्लास्टिक काट्ने बोर्डमा काटिएको तरकारीको गुच्छा छ। एक छेउमा काटिएका च्याउ छन् तिनीहरू सेतो रंग हुन्। अर्को छेउमा केही चम्किलो हरियो कटा ब्रोकाउली छन्। बीचमा उज्यालो सुन्तला काटिएका गाजरहरू छन्।'

In [11]:
clean_vocabulary = []
for txt in data.Paragraph.values:
    clean_vocabulary.extend(txt.split())
print('Clean Vocabulary Size: %d' % len(set(clean_vocabulary)))

Clean Vocabulary Size: 25220


In [12]:
all_captions = []

for caption  in data["Paragraph"].astype(str):
    caption = '<start> ' + caption+ ' <end>'
    all_captions.append(caption)
all_captions[:2]

['<start> यसको अगाडि झ्यालहरूमा बारहरू भएको ठूलो भवन। भवन अगाडि मानिसहरु हिडिरहेका छन्। भवनको अगाडि एउटा सडक जसमा धेरै कारहरू छन्। <end>',
 '<start> एउटा सेतो गोलो प्लेट एउटा टेबलमा जसमा प्लास्टिकको कपडा छ। सेतो प्लेटमा सुनको पहेँलो फ्रेन्च फ्राइजको परिकारसँगै पन्नीले ढाकिएको खानाको दुई भागहरू छन्। छोटो टपलेस प्लास्टिक कन्टेनरमा सेतो प्लेटको छेउमा सेतो चटनी छ। सेतो प्लेटमा विकर्ण धेरै अन्य स्ट्याक्ड प्लेटहरूको किनारहरू हुन्। कालो छायाहरू टेबलमा प्रतिबिम्बित छन्। <end>']

In [13]:
PATH = "/kaggle/input/standford-paragraph-nepali-dataset/stanford_images/"
all_img_name_vector = []

for annot in data["Image_name"]:
    full_image_path = PATH + str(annot)+'.jpg'

    all_img_name_vector.append(full_image_path)
all_img_name_vector[:2]

['/kaggle/input/standford-paragraph-nepali-dataset/stanford_images/2356347.jpg',
 '/kaggle/input/standford-paragraph-nepali-dataset/stanford_images/2317429.jpg']

In [14]:
print(f"len(all_img_name_vector) : {len(all_img_name_vector)}")
print(f"len(all_captions) : {len(all_captions)}")

len(all_img_name_vector) : 20359
len(all_captions) : 20359


In [15]:
def data_limiter(num,total_captions,all_img_name_vector):
  train_captions, img_name_vector = shuffle(total_captions,all_img_name_vector,random_state=1)
  train_captions = train_captions[:num]
  img_name_vector = img_name_vector[:num]
  return train_captions,img_name_vector

In [16]:
train_captions,img_name_vector = data_limiter(19561,all_captions,all_img_name_vector)

In [17]:
print(f"len(all_img_name_vector) : {len(img_name_vector)}")
print(f"len(all_captions) : {len(train_captions)}")

len(all_img_name_vector) : 19561
len(all_captions) : 19561


In [18]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [19]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

87910968/87910968 [==============================] - 3s 0us/step


In [ ]:
# Get unique images
import os
import h5py

encode_train = sorted(set(img_name_vector))

# Create lists to store all features and paths
all_features = []
all_paths = []

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
    load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                                (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
        all_features.append(bf.numpy())
        all_paths.append(p.numpy().decode("utf-8"))

# Convert lists to NumPy arrays
all_features = np.array(all_features)

# Save features and paths to HDF5 file
output_file = '/kaggle/working/all_image_features.h5'

with h5py.File(output_file, 'w') as hf:
    hf.create_dataset('features', data=all_features)
    hf.create_dataset('paths', data=np.array(all_paths, dtype='S'))  # 'S' is for string dtype


 61%|██████    | 748/1223 [02:42<01:42,  4.63it/s]

In [36]:
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import os

npic = 5
npix = 224
target_size = (npix, npix, 3)

count = 1
fig = plt.figure(figsize=(10, 20))

# Assuming you have loaded the Stanford Paragraph Captioning dataset into 'data'
# and set the appropriate path to the image directory
image_path = '/kaggle/input/standford-paragraph-nepali-dataset/stanford_images'

# Extract unique filenames from the dataset
uni_filenames = data["Image_name"].unique()

# for jpgfnm in uni_filenames[:5]:
#     filename = os.path.join(image_path, str(jpgfnm)+'.jpg')  # Convert jpgfnm to string
#     captions = list(data["Paragraph"].loc[data["Image_name"] == jpgfnm].values)

#     image_load = load_img(filename, target_size=target_size)

#     ax = fig.add_subplot(npic, 2, count, xticks=[], yticks=[])
#     ax.imshow(image_load)
#     count += 1

#     ax = fig.add_subplot(npic, 2, count)
#     plt.axis('off')
#     ax.plot()
#     ax.set_xlim(0, 1)
#     ax.set_ylim(0, len(captions))
#     for i, caption in enumerate(captions):
#         ax.text(0, i, caption, fontsize=10, fontproperties=nepali_font)   # Adjusted fontsize
#     count += 1

# plt.show()


<Figure size 1000x2000 with 0 Axes>